# HP 8722D pyVNA tutorial

In [1]:
# Import the things we need - pyVNA is the package of interest.
# You might need to add its location to path manually, but not in this case.
import sys,time,struct,math,numpy as np,matplotlib.pyplot as plt
from timeit import default_timer as timer
#sys.path.append('/home/pi/Desktop/ADRNX/VNA')
from pyVNA import core,constants

In [ ]:
# Initialize vna object, overrding some defaults. Debug flag turns on/off print statements.
vna = core.VNA(dev='/dev/ttyUSB2',debug=False)
# Set up gpib adapter comms
vna.init_and_setup()
# Preset VNA to factory defaults - command blocks until this completes or timeout of 5s
vna.preset(tm=5)

** Changing measurement type and mode **

A special class, ``constants``, is provided for convenient enumeration of possible mode and measurement values
For instance: '``constants.VNA_MODE.POLAR``'

In [ ]:
# First check if desired mode or measurement type is on(true)
# Only VNA_MODE and VNA_MEAS constants are valid arguments
print(vna.check_if_true(constants.VNA_MODE.POLAR),vna.check_if_true(constants.VNA_MEAS.S21))

In [ ]:
# Set things to what we want
vna.set_meas_mode(constants.VNA_MEAS.S21)
time.sleep(0.1)
vna.set_plot_mode(constants.VNA_MODE.POLAR)

In [ ]:
# Verify that changes did go through (wait a bit for things to switch)
print(vna.check_if_true(constants.VNA_MODE.POLAR),vna.check_if_true(constants.VNA_MEAS.S21))

** Changing sweep parameters **

In [ ]:
# Set setting to what we want, printing state after each change
vna.print_current_state()
vna.set_ifbw(30)
vna.print_current_state()
vna.set_num_points(1601)
vna.print_current_state()

fmin, fmax = 2.275E3, 2.276E3

print(fmin, fmax)

vna.set_freq_range(fmin, fmax,units="MHz")
vna.print_current_state()
vna.set_power(-10)
vna.print_current_state()

** Doing a sweep **

In [ ]:
# After setting desired parameters, several method will use those to give useful sweep properties
# For example, we can get a rough local estimate of sweep time (quickly, for algorithm scheduling purposes)
print(vna.estimate_sweep_time())
# Or query VNA for actual value (which is by default the lowest possible sweep time for current settings)
swt = vna.get_actual_sweep_time()
print(swt)
# And also get number of bytes we expect from VNA
# Note how result of form 2 (binary transfer) is so much smaller than ASCII form 4
# (don't worry - parsing of these is handled internally, so this is just for your memory consumption calcs)
print(vna.estimate_sweep_size(form=2),vna.estimate_sweep_size(form=4))

In [ ]:
# Now, lets start the sweep - this command returns immediately, and you can do other things while vna sweeps
# Note how sweep params dictionary is generated at this point
sweep_parameters = vna.start_sweep()
# But here, we will block to await completion with a timeout (returns -1 if timeout occurs)
timetaken = vna.await_completion(swt+10.0)
print(timetaken)

** Reading out data **

In [ ]:
# Now, we need to explicitly read out the data.
# This can take awhile if run immediately after sweep, since vna is formatting things internally.
start = timer()
data2 = vna.retrieve_sweep_data(pause=0,form=2)
print(timer()-start)
# You can read out same data many times. Note how ASCII readout takes longer and is generally discouraged.
start = timer()
data4 = vna.retrieve_sweep_data(pause=0,form=4)
print(timer()-start)

In [ ]:
# Finally, we can at a later timer parse data blob + sweep parameters to obtain results dictionary
results = vna.parse_vna_data(sweep_parameters, data2, form=2)
results.keys()

In [ ]:
# Let's plot it!
[il,ql] = [results["I"],results["Q"]]
datatoplot = [10.0*math.log10((il[i]**2+ql[i]**2)/(50.0*.001)) for i in range(0,len(il))]
plt.figure()
plt.plot(results["F"],datatoplot,'-',lw=2)
plt.show()

** Saving data **

In [ ]:
# Assuming you want to mimic the standard output, utilities are provided to create text files from results dictionary
# You can either specify folder (which exists) (and use autonaming 'vnasweep-%Y-%m-%d-%H-%M-%S.txt')
vna.save_results_to_file(results,folder='/home/pi/Desktop/DataNX/vna/tests/')
# Or, specify a full path
vna.save_results_to_file(results,fullpath='/home/pi/Desktop/DataNX/vna/tests/test.blaa')

** But what if I am lazy and want 1 liners? **

In [ ]:
results = vna.do_full_measurement({"ifbw": 30, "numpoints": 1601, "freqs": [500,700,"MHz"], "power": -8,\
                               "mode": constants.VNA_MODE.POLAR, "meas": constants.VNA_MEAS.S21, "preset": True})

# its two lines...close enough!

In [ ]:
# Let's plot it (again)!
[il,ql] = [results["I"],results["Q"]]
datatoplot = [10.0*math.log10((il[i]**2+ql[i]**2)/(50.0*.001)) for i in range(0,len(il))]
plt.figure()
plt.plot(results["F"],datatoplot,'-')
plt.show()